In [ ]:
#can't make progress on this with out ground zero points w/ logical and euc grid distance 

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timezone
from datetime import timedelta
import pyproj
import seaborn as sns
import geopandas as gpd 
import shapely.geometry as geometry
import shapely.ops as ops 
from functools import partial 

In [11]:
#now let's do this analysis with the logical grid distance as our distance metric 
logical = pd.read_csv('/Users/emilypaszkiewicz17/gridwatch-data-analysis/emily_analysis/grid_distance.csv')
logical = logical.reset_index()
#these are the transition points of the outages 
outages = pd.read_parquet('part-00000-3c7aa0ea-41c7-4705-bafc-5662f2051563-c000.gz.parquet')
outages.head()

In [ ]:
#I've commented out lines that will probably need to be changed when implementing logical/euclidean grid distance 
#there are probably more, but start there 


class STDBSCAN(object):

    def __init__(self, col_lat, col_lon, col_time, spatial_threshold=500.0, 
                 temporal_threshold=60.0, min_neighbors=15):
        """
        Python st-dbscan implementation.
        :param col_lat: Latitude column name;
        :param col_lon:  Longitude column name;
        :param col_time: Date time column name;
        :param spatial_threshold: Maximum geographical coordinate (spatial)
             distance value (meters);
        :param temporal_threshold: Maximum non-spatial distance value (seconds);
        :param min_neighbors: Minimum number of points within Eps1 and Eps2
             distance;
        """
        self.col_lat = col_lat
        self.col_lon = col_lon
        self.col_time = col_time
        self.spatial_threshold = spatial_threshold
#         self.temporal_threshold = temporal_threshold
        self.min_neighbors = min_neighbors

#     def projection(self, df, p1_str='epsg:4326', p2_str='epsg:3395'):
#         """
#         Cython wrapper to converts from geographic (longitude,latitude)
#         to native map projection (x,y) coordinates. It needs to select the
#         right epsg. Values of x and y are given in meters
#         """
#         p1 = pyproj.Proj(init=p1_str)
#         p2 = pyproj.Proj(init=p2_str)
#         lon = df[self.col_lon].values
#         lat = df[self.col_lat].values
#         x1, y1 = p1(lon, lat)
#         x2, y2 = pyproj.transform(p1, p2, x1, y1, radians=True)
#         df[self.col_lon] = x2
#         df[self.col_lat] = y2

#         print(df)
#         return df

    def _retrieve_neighbors(self, index_center, matrix):

        center_point = matrix[index_center, :]

        # filter by time
        min_time = center_point[2] - timedelta(seconds=self.temporal_threshold)
        max_time = center_point[2] + timedelta(seconds=self.temporal_threshold)
        matrix = matrix[(matrix[:, 2] >= min_time) &
                        (matrix[:, 2] <= max_time), :]
        # filter by distance
        
        
        tmp = 
#THIS is where the algorithm computes euclidean distance and compares it to the threshold 

#         (matrix[:, 0]-center_point[0])*(matrix[:, 0]-center_point[0]) + \
#             (matrix[:, 1]-center_point[1])*(matrix[:, 1]-center_point[1])
        
        neigborhood = matrix[tmp <= (
            self.spatial_threshold*self.spatial_threshold), 4].tolist()
        
        
        
        
        
        neigborhood.remove(index_center)

        return neigborhood

    def run(self, df):
        """
        INPUTS:
            df={o1,o2,...,on} Set of objects;
        OUTPUT:
            C = {c1,c2,...,ck} Set of clusters
        """
        cluster_label = 0
        noise = -1
        unmarked = 777777
        stack = []

        # initial setup
#         df = df[[self.col_lon, self.col_lat, self.col_time]]
        df = df.assign(cluster=unmarked)
        df['index'] = range(df.shape[0])
        matrix = df.values
        df.drop(['index'], inplace=True, axis=1)

        # for each point in database
        for index in range(matrix.shape[0]):
            if matrix[index, 3] == unmarked:
                neighborhood = self._retrieve_neighbors(index, matrix)

                if len(neighborhood) < self.min_neighbors:
                    matrix[index, 3] = noise
                else:  # found a core point
                    cluster_label += 1
                    # assign a label to core point
                    matrix[index, 3] = cluster_label

                    # assign core's label to its neighborhood
                    for neig_index in neighborhood:
                        matrix[neig_index, 3] = cluster_label
                        stack.append(neig_index)  # append neighbors to stack

                    # find new neighbors from core point neighborhood
                    while len(stack) > 0:
                        current_point_index = stack.pop()
                        new_neighborhood = \
                            self._retrieve_neighbors(current_point_index,
                                                     matrix)

                        # current_point is a new core
                        if len(new_neighborhood) >= self.min_neighbors:
                            for neig_index in new_neighborhood:
                                neig_cluster = matrix[neig_index, 3]
                                if any([neig_cluster == noise,
                                        neig_cluster == unmarked]):
                                    matrix[neig_index, 3] = cluster_label
                                    stack.append(neig_index)

        df['cluster'] = matrix[:, 3]
        return df

In [ ]:
def parse_dates(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')

def plot_clusters(df, output_name):
    import matplotlib.pyplot as plt

    labels = df['cluster'].values
    X = df[['longitude', 'latitude']].values

    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)

    plt.title('ST-DSCAN: #n of clusters {}'.format(len(unique_labels)))
    plt.show()
    # plt.savefig(output_name)


def test_time(df):
    '''
    transfrom the lon and lat to x and y
    need to select the right epsg
    I don't the true epsg of sample, but get the same result by using 
    epsg:4326 and epsg:32635
    '''
    st_dbscan = STDBSCAN(col_lat='location_latitude', col_lon='location_longitude',
                         col_time='time', spatial_threshold=0.03,
                         temporal_threshold=60, min_neighbors=1)
    #df = st_dbscan.projection(df, p1_str='epsg:4326', p2_str='epsg:32630')
    result_t180 = st_dbscan.run(df)
    return result_t180

